# OpenAI Responses API: Complete Tutorial

## Overview

The OpenAI Responses API is a superset of Chat Completions, meaning everything you can do with Chat Completions can be done with the Responses API, plus additional features. This unified interface simplifies building AI applications with built-in tools, conversation management, and enhanced capabilities.

### Key Features
- **Simplified Interface**: Single API for complex interactions
- **Built-in Tools**: Web search, file search, computer use
- **Stateful Conversations**: Automatic conversation management
- **Structured Outputs**: Type-safe responses with Pydantic
- **Enhanced Function Calling**: Custom tool integration

## Setup

In [ ]:
# Install required packages
!pip install openai python-dotenv

In [ ]:
import os
from openai import OpenAI
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()

# Initialize client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

## 1. Basic Text Generation

In [ ]:
# Simple text generation
response = client.responses.create(
    model="gpt-4o",
    input="Write a haiku about artificial intelligence."
)

print(response.output_text)

### Instructions vs Input

In [ ]:
# Using instructions for system-level guidance
response = client.responses.create(
    model="gpt-4o",
    instructions="You are a helpful coding assistant. Always provide working code examples.",
    input="How do I read a CSV file in Python?"
)

print(response.output_text)

## 2. Conversation State Management

### Manual State Management

In [ ]:
# Build conversation manually
conversation = [
    {"role": "user", "content": "What's the capital of France?"},
    {"role": "assistant", "content": "The capital of France is Paris."},
    {"role": "user", "content": "What's the population?"}
]

response = client.responses.create(
    model="gpt-4o-mini",
    input=conversation
)

print(response.output_text)

### Automatic State Management

In [ ]:
# Let OpenAI handle conversation state
response1 = client.responses.create(
    model="gpt-4o-mini",
    input="Tell me about Python programming."
)

# Continue conversation using previous response ID
response2 = client.responses.create(
    model="gpt-4o-mini",
    previous_response_id=response1.id,
    input="What are its main advantages?"
)

print("First response:", response1.output_text)
print("\nSecond response:", response2.output_text)

## 3. Function Calling

In [ ]:
# Define custom functions
tools = [
    {
        "type": "function",
        "name": "get_weather",
        "description": "Get current weather for a location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {"type": "string", "description": "City name"},
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]}
            },
            "required": ["location"]
        }
    }
]

response = client.responses.create(
    model="gpt-4o",
    input="What's the weather like in Tokyo?",
    tools=tools
)

# Check if function was called
for output in response.output:
    if hasattr(output, 'function_call'):
        print(f"Function called: {output.function_call.name}")
        print(f"Arguments: {output.function_call.arguments}")

## 4. Structured Outputs

In [ ]:
from pydantic import BaseModel
from typing import List

# Define response structure
class EventDetails(BaseModel):
    name: str
    date: str
    participants: List[str]
    location: str

# Generate structured output
response = client.responses.create(
    model="gpt-4o",
    input="Alice and Bob are meeting for a coding workshop at the library on Friday.",
    text={
        "format": {
            "type": "json_schema",
            "name": "event_extraction",
            "schema": EventDetails.model_json_schema(),
            "strict": True
        }
    }
)

# Parse structured response
event_data = json.loads(response.output_text)
print(json.dumps(event_data, indent=2))

## 5. Built-in Tools

### Web Search

In [ ]:
# Use web search tool
response = client.responses.create(
    model="gpt-4o",
    input="What are the latest developments in AI in 2025?",
    tools=[{"type": "web_search_preview"}]
)

print(response.output_text)

# Access citations
for output in response.output:
    if hasattr(output, 'content'):
        for content in output.content:
            if hasattr(content, 'annotations'):
                for annotation in content.annotations:
                    if annotation.type == 'url_citation':
                        print(f"Source: {annotation.title}")
                        print(f"URL: {annotation.url}")

### File Search

In [ ]:
# Upload and search files (requires vector store setup)
# First, upload a file
# with open("sample_document.pdf", "rb") as file:
#     uploaded_file = client.files.create(file=file, purpose="assistants")

# Create vector store
# vector_store = client.vector_stores.create(name="knowledge_base")

# Add file to vector store
# client.vector_stores.files.create(
#     vector_store_id=vector_store.id,
#     file_id=uploaded_file.id
# )

# Search within uploaded files
# response = client.responses.create(
#     model="gpt-4o",
#     input="What are the key findings in the uploaded document?",
#     tools=[{
#         "type": "file_search",
#         "vector_store_ids": [vector_store.id],
#         "max_num_results": 3
#     }]
# )

# print(response.output_text)

print("File search example - uncomment code above when you have files to upload")

## 6. Image Analysis

In [ ]:
# Analyze images
response = client.responses.create(
    model="gpt-4o",
    input=[
        {"role": "user", "content": "Describe what you see in this image."},
        {
            "role": "user", 
            "content": [{
                "type": "input_image",
                "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/4/47/PNG_transparency_demonstration_1.png/280px-PNG_transparency_demonstration_1.png"
            }]
        }
    ]
)

print(response.output_text)

## 7. Streaming Responses

In [ ]:
# Stream responses in real-time
stream = client.responses.create(
    model="gpt-4o",
    input="Explain machine learning in simple terms.",
    stream=True
)

print("Streaming response:")
for event in stream:
    if hasattr(event, "type") and "text.delta" in event.type:
        print(event.delta, end="", flush=True)

## 8. Advanced Features

### Background Mode (for reasoning models)

In [ ]:
# Use reasoning models with background processing
response = client.responses.create(
    model="o3-mini",
    input="Solve this complex math problem: Find the derivative of x^3 + 2x^2 - 5x + 3",
    reasoning={"effort": "medium"}
)

print(response.output_text)

### Error Handling

In [ ]:
import openai

try:
    response = client.responses.create(
        model="gpt-4o",
        input="Hello world!"
    )
    print(response.output_text)
    
except openai.RateLimitError:
    print("Rate limit exceeded. Please try again later.")
except openai.APIError as e:
    print(f"API error occurred: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

## Best Practices

1. **Use appropriate models**: Choose `gpt-4o` for complex tasks, `gpt-4o-mini` for simple operations
2. **Manage conversation state**: Use `previous_response_id` for multi-turn conversations
3. **Leverage built-in tools**: Web search and file search reduce development complexity
4. **Structure outputs**: Use JSON schemas for consistent, parseable responses
5. **Handle errors gracefully**: Implement proper error handling for production applications
6. **Monitor usage**: Track API usage and costs in production environments

## Migration from Chat Completions

The Responses API is backward compatible with Chat Completions, making migration straightforward:

In [ ]:
# Old Chat Completions way
# response = client.chat.completions.create(
#     model="gpt-4o",
#     messages=[{"role": "user", "content": "Hello!"}]
# )

# New Responses API way
response = client.responses.create(
    model="gpt-4o",
    input="Hello!"
)

print(response.output_text)

## Conclusion

The Responses API provides a more powerful, unified interface for building sophisticated AI applications with less code and complexity. It combines orchestration logic and integrates built-in tools without requiring custom implementation, making it the preferred choice for new projects.

Key advantages:
- Simplified development experience
- Built-in tools reduce boilerplate code
- Automatic conversation state management
- Enhanced function calling capabilities
- Support for multimodal inputs and structured outputs